In [2]:
import numpy as np
import random, math
import copy

Dimension = 2
qubits = 2
A = np.array([[3, 1], [-1, 2]])
b = np.array([-1, 5])

# 8 qubits with integer solutions
# x = q1 + 2q2 - q3 - 2q4
# x = {{-1}, {2}}
# mininum -26

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

### q+ * q- 
cstrd = 500
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits):
            for l2 in range(qubits):
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2 + qubits
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                QM[po1][po2] = QM[po1][po2] - qcef + cstrd

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[ 26.  40. 980. 960.   2.   4.  -2.  -4.]
 [  0.  72. 960. 920.   4.   8.  -4.  -8.]
 [  0.   0.  -6.  40.  -2.  -4.   2.   4.]
 [  0.   0.   0.   8.  -4.  -8.   4.   8.]
 [  0.   0.   0.   0. -13.  20. 990. 980.]
 [  0.   0.   0.   0.   0. -16. 980. 960.]
 [  0.   0.   0.   0.   0.   0.  23.  20.]
 [  0.   0.   0.   0.   0.   0.   0.  56.]]

Minimum energy is  -26


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q3'):980.0, ('q1','q4'):960.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q3'):960.0, ('q2','q4'):920.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q

In [3]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q3'):980.0, ('q1','q4'):960.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q3'):960.0, ('q2','q4'):920.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q5','q7'):990.0, ('q5','q8'):980.0, ('q6','q7'):980.0, ('q6','q8'):960.0, ('q7','q8'):20.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      22     0.0
1   0  0  1  0  1  0  0  0  -21.0       8     0.0
2   0  0  1  0  1  1  0  0  -21.0      39     0.0
3   0  0  0  0  0  1  0  0  -16.0      14     0.0
4   0  0  0  1  0  1  0  0  -16.0      34     0.0
5   0  0  0  1  1  1  0  0  -13.0      39     0.0
6   0  0  0  0  1  0  0  0  -13.0       9     0.0
7   0  0  0  0  1  1  0  0   -9.0      20     0.0
8   0  0  0  1  1  0  0  0   -9.0      24     0.0
9   0  0  1  0  0  0  0  0   -6.0      15     0.0
10  0  0  0  0  0  0  0  0    0.0      12     0.0
11  0  0  0  1  0  0  0  0    8.0      17     0.0
12  1  0  0  0  0  1  0  0   14.0      15     0.0
13  0  0  1  1  0  1  0  0   14.0      30     0.0
14  1  0  0  0  1  0  0  0   15.0       8     0.0
15  0  0  1  1  1  1  0  0   15.0      52     0.0
16  0  0  1  0  0  0  1  0   19.0      31     0.0
17  0  0  0  0  0  0  1  0   23.0      29     0.0
18  1  0  0  0  1  1  0  0   23.0      12     0.0


In [4]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      44     0.0
50  0  0  1  0  0  1  0  0  -26.0       1   0.125
1   0  0  1  0  1  1  0  0  -21.0     142     0.0
2   0  0  1  0  1  0  0  0  -21.0      68     0.0
3   0  0  0  1  0  1  0  0  -16.0      17     0.0
4   0  0  0  0  0  1  0  0  -16.0      14     0.0
5   0  0  0  0  1  0  0  0  -13.0      29     0.0
6   0  0  0  1  1  1  0  0  -13.0      58     0.0
49  0  0  0  0  1  0  0  0  -13.0       1   0.125
7   0  0  0  0  1  1  0  0   -9.0      45     0.0
8   0  0  0  1  1  0  0  0   -9.0      33     0.0
9   0  0  1  0  0  0  0  0   -6.0      22     0.0
10  0  0  0  0  0  0  0  0    0.0       5     0.0
11  0  0  0  1  0  0  0  0    8.0       6     0.0
12  0  0  1  1  0  1  0  0   14.0      26     0.0
13  1  0  0  0  0  1  0  0   14.0      21     0.0
14  1  0  0  0  1  0  0  0   15.0      27     0.0
15  0  0  1  1  1  1  0  0   15.0     105     0.0
16  0  0  1  0  0  0  1  0   19.0      28     0.0


In [5]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      55     0.0
1   0  0  1  0  1  1  0  0  -21.0     104     0.0
2   0  0  1  0  1  0  0  0  -21.0      38     0.0
3   0  0  0  0  0  1  0  0  -16.0      16     0.0
4   0  0  0  1  0  1  0  0  -16.0      11     0.0
5   0  0  0  1  1  1  0  0  -13.0      41     0.0
6   0  0  0  0  1  0  0  0  -13.0      15     0.0
7   0  0  0  0  1  1  0  0   -9.0      26     0.0
8   0  0  0  1  1  0  0  0   -9.0      12     0.0
9   0  0  1  0  0  0  0  0   -6.0      34     0.0
10  0  0  0  0  0  0  0  0    0.0      14     0.0
49  0  0  0  0  0  0  0  0    0.0       1   0.125
11  0  0  0  1  0  0  0  0    8.0       9     0.0
12  1  0  0  0  0  1  0  0   14.0      26     0.0
13  0  0  1  1  0  1  0  0   14.0      49     0.0
14  0  0  1  1  1  1  0  0   15.0      79     0.0
15  1  0  0  0  1  0  0  0   15.0      20     0.0
16  0  0  1  0  0  0  1  0   19.0      46     0.0
17  0  0  1  1  1  0  0  0   23.0      26     0.0


In [6]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      11     0.0
1   0  0  1  0  1  1  0  0  -21.0      53     0.0
2   0  0  1  0  1  0  0  0  -21.0      19     0.0
3   0  0  0  1  0  1  0  0  -16.0      44     0.0
4   0  0  0  0  0  1  0  0  -16.0      16     0.0
5   0  0  0  1  1  1  0  0  -13.0     127     0.0
6   0  0  0  0  1  0  0  0  -13.0      16     0.0
7   0  0  0  1  1  0  0  0   -9.0      64     0.0
8   0  0  0  0  1  1  0  0   -9.0      43     0.0
9   0  0  1  0  0  0  0  0   -6.0       3     0.0
10  0  0  0  0  0  0  0  0    0.0       2     0.0
11  0  0  0  1  0  0  0  0    8.0      17     0.0
12  0  0  1  1  0  1  0  0   14.0      32     0.0
13  1  0  0  0  0  1  0  0   14.0      31     0.0
14  1  0  0  0  1  0  0  0   15.0      41     0.0
15  0  0  1  1  1  1  0  0   15.0      89     0.0
16  0  0  1  0  0  0  1  0   19.0      10     0.0
17  1  0  0  0  1  1  0  0   23.0      82     0.0
18  0  0  1  1  1  0  0  0   23.0      30     0.0


In [7]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      11     0.0
1   0  0  1  0  1  0  0  0  -21.0      33     0.0
2   0  0  1  0  1  1  0  0  -21.0      31     0.0
3   0  0  0  0  0  1  0  0  -16.0      13     0.0
4   0  0  0  1  0  1  0  0  -16.0      18     0.0
5   0  0  0  0  1  0  0  0  -13.0      25     0.0
6   0  0  0  1  1  1  0  0  -13.0      29     0.0
7   0  0  0  0  1  1  0  0   -9.0      20     0.0
8   0  0  0  1  1  0  0  0   -9.0      23     0.0
9   0  0  1  0  0  0  0  0   -6.0      24     0.0
10  0  0  0  0  0  0  0  0    0.0      11     0.0
11  0  0  0  1  0  0  0  0    8.0      22     0.0
12  0  0  1  1  0  1  0  0   14.0      18     0.0
13  1  0  0  0  0  1  0  0   14.0      40     0.0
14  1  0  0  0  1  0  0  0   15.0      35     0.0
15  0  0  1  1  1  1  0  0   15.0      29     0.0
16  0  0  1  0  0  0  1  0   19.0      28     0.0
17  0  0  1  1  1  0  0  0   23.0      24     0.0
18  0  0  0  0  0  0  1  0   23.0      18     0.0


In [8]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      51     0.0
1   0  0  1  0  1  0  0  0  -21.0      27     0.0
2   0  0  1  0  1  1  0  0  -21.0      84     0.0
3   0  0  0  0  0  1  0  0  -16.0      29     0.0
4   0  0  0  1  0  1  0  0  -16.0      33     0.0
49  0  0  0  0  0  1  0  0  -16.0       1   0.125
5   0  0  0  0  1  0  0  0  -13.0      28     0.0
6   0  0  0  1  1  1  0  0  -13.0      38     0.0
7   0  0  0  1  1  0  0  0   -9.0      15     0.0
8   0  0  0  0  1  1  0  0   -9.0      32     0.0
50  0  0  0  0  1  1  0  0   -9.0       1   0.125
53  0  0  0  0  1  1  0  0   -9.0       1   0.125
9   0  0  1  0  0  0  0  0   -6.0      13     0.0
10  0  0  0  0  0  0  0  0    0.0      11     0.0
11  0  0  0  1  0  0  0  0    8.0       9     0.0
12  0  0  1  1  0  1  0  0   14.0      40     0.0
13  1  0  0  0  0  1  0  0   14.0      42     0.0
14  1  0  0  0  1  0  0  0   15.0      35     0.0
15  0  0  1  1  1  1  0  0   15.0      70     0.0


In [9]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      47     0.0
1   0  0  1  0  1  1  0  0  -21.0      56     0.0
2   0  0  1  0  1  0  0  0  -21.0      20     0.0
3   0  0  0  0  0  1  0  0  -16.0      20     0.0
4   0  0  0  1  0  1  0  0  -16.0      72     0.0
5   0  0  0  1  1  1  0  0  -13.0     152     0.0
6   0  0  0  0  1  0  0  0  -13.0      15     0.0
7   0  0  0  1  1  0  0  0   -9.0      22     0.0
8   0  0  0  0  1  1  0  0   -9.0      48     0.0
9   0  0  1  0  0  0  0  0   -6.0      10     0.0
10  0  0  0  0  0  0  0  0    0.0       4     0.0
11  0  0  0  1  0  0  0  0    8.0       7     0.0
12  1  0  0  0  0  1  0  0   14.0      35     0.0
13  0  0  1  1  0  1  0  0   14.0      73     0.0
14  0  0  1  1  1  1  0  0   15.0     100     0.0
15  1  0  0  0  1  0  0  0   15.0      20     0.0
16  0  0  1  0  0  0  1  0   19.0       6     0.0
17  0  0  0  0  0  0  1  0   23.0       5     0.0
18  1  0  0  0  1  1  0  0   23.0      54     0.0


In [10]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      44     0.0
44  0  0  1  0  0  1  0  0  -26.0       1   0.125
1   0  0  1  0  1  1  0  0  -21.0      57     0.0
2   0  0  1  0  1  0  0  0  -21.0      16     0.0
45  0  0  1  0  1  1  0  0  -21.0       1   0.125
3   0  0  0  0  0  1  0  0  -16.0      30     0.0
4   0  0  0  1  0  1  0  0  -16.0      66     0.0
5   0  0  0  1  1  1  0  0  -13.0      90     0.0
6   0  0  0  0  1  0  0  0  -13.0       6     0.0
7   0  0  0  1  1  0  0  0   -9.0      27     0.0
8   0  0  0  0  1  1  0  0   -9.0      28     0.0
9   0  0  1  0  0  0  0  0   -6.0      19     0.0
10  0  0  0  0  0  0  0  0    0.0       5     0.0
11  0  0  0  1  0  0  0  0    8.0      18     0.0
12  0  0  1  1  0  1  0  0   14.0     113     0.0
13  1  0  0  0  0  1  0  0   14.0      13     0.0
14  0  0  1  1  1  1  0  0   15.0     136     0.0
15  1  0  0  0  1  0  0  0   15.0       2     0.0
16  0  0  1  0  0  0  1  0   19.0      19     0.0


In [11]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      37     0.0
1   0  0  1  0  1  0  0  0  -21.0      56     0.0
2   0  0  1  0  1  1  0  0  -21.0      79     0.0
3   0  0  0  1  0  1  0  0  -16.0      11     0.0
4   0  0  0  0  0  1  0  0  -16.0      12     0.0
5   0  0  0  0  1  0  0  0  -13.0      18     0.0
6   0  0  0  1  1  1  0  0  -13.0      37     0.0
7   0  0  0  1  1  0  0  0   -9.0      31     0.0
8   0  0  0  0  1  1  0  0   -9.0      29     0.0
49  0  0  0  0  1  1  0  0   -9.0       1   0.125
9   0  0  1  0  0  0  0  0   -6.0      33     0.0
10  0  0  0  0  0  0  0  0    0.0       5     0.0
11  0  0  0  1  0  0  0  0    8.0      14     0.0
12  1  0  0  0  0  1  0  0   14.0       5     0.0
13  0  0  1  1  0  1  0  0   14.0      26     0.0
14  1  0  0  0  1  0  0  0   15.0      23     0.0
15  0  0  1  1  1  1  0  0   15.0      68     0.0
16  0  0  1  0  0  0  1  0   19.0      42     0.0
17  0  0  1  1  1  0  0  0   23.0      66     0.0


In [12]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  0  1  0  0  -26.0      26     0.0
1   0  0  1  0  1  0  0  0  -21.0      66     0.0
2   0  0  1  0  1  1  0  0  -21.0      30     0.0
3   0  0  0  1  0  1  0  0  -16.0      12     0.0
4   0  0  0  0  0  1  0  0  -16.0       8     0.0
5   0  0  0  0  1  0  0  0  -13.0      18     0.0
6   0  0  0  1  1  1  0  0  -13.0      21     0.0
7   0  0  0  0  1  1  0  0   -9.0      11     0.0
8   0  0  0  1  1  0  0  0   -9.0      28     0.0
9   0  0  1  0  0  0  0  0   -6.0      32     0.0
10  0  0  0  0  0  0  0  0    0.0      10     0.0
11  0  0  0  1  0  0  0  0    8.0      30     0.0
12  1  0  0  0  0  1  0  0   14.0       6     0.0
13  0  0  1  1  0  1  0  0   14.0      34     0.0
14  1  0  0  0  1  0  0  0   15.0      15     0.0
15  0  0  1  1  1  1  0  0   15.0      59     0.0
49  0  0  1  1  1  1  0  0   15.0       1   0.125
16  0  0  1  0  0  0  1  0   19.0      45     0.0
17  1  0  0  0  1  1  0  0   23.0       9     0.0
